Copyright (c) 2024, Krista Dotterer
All rights reserved.

This source code is licensed under the BSD-style license found in the
LICENSE file in the root directory of this source tree. 

In [774]:
#import important packages
import numpy as np
import xarray as xr
import math
import scipy
import pandas as pd
import time as tm
import netCDF4 as n
import matplotlib.pyplot as plt
import matplotlib.colors as colors

'''
Current code is written for the following coordinates:
-Latitude: south to north
-Longitude: 0 to 360 degrees
(Coordinates of CLAUS-IR merged dataset)

If data is north to south or -180 to 180 degrees, the code may break. The easiest fix is to restructure your data
before tracking. 

This code is untested for any non-convective CCKW indicator. It is designed for Kelvin-filtered OLR or Tb. 


'''

In [776]:
# Some functions
# Sourced from AEW tracker written by Quinton Lawton with permission

#Finds the nearest longitude or latitude in dataset
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    #print(idx)
    return idx, array[idx]

def haversine(lon1, lat1, lon2, lat2):
    from math import radians, cos, sin, asin, sqrt
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km
def within_distance_direct(lon, lat, lon_in,lat_in, lon_old, lat_old, step, forward_weight = True, forward_scale = 1/5):
    '''Right now, we want centers to be within a distance of the time step. 
    
    Forward bias prevents extreme cases of "back_building" by limiting back_lon search to half the forward distance.'''
    if lon_in <= lon_old: #IF true, then the wave is moving the correct direction
        fwd = True
    else:
        fwd = False
    distance = haversine(lon_in, lat_in, lon_old, lat_old)
    if distance <= step:
        return True, distance, fwd
    else: 
        return False, distance, fwd
    
def rad_mask(i, j, dx, dy, radius):
    '''This is a computationally efficient (at least, in python-world) way of calculating a mask of gridpoints around a
    center point. This generates an array of points with values corresponding to the distance from some center point. 
    Then the code masks out any values above a certain radius. 
    
    This uses the assumption of a spherical Earth, not accounting for the equatorial "bulge" in real life.'''
    start = tm.time()

    earth_circ = 6371*2*np.pi*1000 #earth's circumference in meters
    lat_met = earth_circ/360 #get the number of meters in a degree latitude (ignoring "bulge")
    lat_km = lat_met/1000
    res = 1
    buffer = int(np.ceil(radius/lat_km/res))
    boolean_array = np.zeros(np.shape(dx), dtype=bool)
    #i_array = np.zeros(np.shape(dy))
    #j_array = np.zeros(np.shape(dx))
    dy = -dy

    # Before doing this, we recognize something -- there is a max radius in the x and y direction that will
    # be computed. To save on computational time, we need to slice out the extraneous points that we already
    # know from this NOT to be in the circle. How? We know the absolute highest distance between gridspaces
    # will be the maximum value of latitude on the elliptical earth... ~ 111 km. So slice out roughly a 112km "box" 
    # plus one gridbox for buffer. 
    
    i_st = (i-(buffer+1))
    i_end = (i+(buffer+1))
    j_st = (j-(buffer+1))
    j_end = (j+(buffer+1))
    
    if i_st <0:
        i_st = 0 
    new_i = i-i_st
    new_j = j-j_st
    
    
    dy_slc = dy[i_st:i_end,j_st:j_end]
    dx_slc = dx[i_st:i_end,j_st:j_end]
    
    
    i_array_sub = np.zeros(np.shape(dy_slc))
    j_array_sub = np.zeros(np.shape(dx_slc))
    
    i_array_sub[new_i,:] = 0
    i_array_sub[(new_i+1):,:] = np.add.accumulate(dy_slc[(new_i+1):,:])
    i_array_sub[(new_i-1)::-1,:] = np.add.accumulate(dy_slc[(new_i-1)::-1,:])
    
    j_array_sub[:,new_j] = 0
    j_array_sub[:,(new_j+1):] = np.add.accumulate(dx_slc[:,(new_j+1):], axis=1)
    j_array_sub[:,(new_j-1)::-1] = np.add.accumulate(dx_slc[:,(new_j-1)::-1], axis=1)
    
    radial_array = (np.sqrt(np.square(i_array_sub)+np.square(j_array_sub))/1000) < radius # in km
    boolean_array[i_st:i_end,j_st:j_end] = radial_array
    
    end = tm.time()
    #print(end - start)
    return boolean_array

In [777]:
# String of year of data to be tracked
# Currently only used in data import and export. 
year = '1991'

In [778]:
# Import data
claus = xr.open_dataset('/thorncroftlab_rit/kd829281/Data/CLAUSTbKelvHarmAnom'+year+'_test.nc')

In [761]:
#Converts data from -180 - 180 to 0 - 360
#Uncomment if necessary
#claus = claus.assign_coords(lon=(claus.lon % 360)).sortby('lon')

In [779]:
claus

<xarray.Dataset>
Dimensions:  (time: 2920, lat: 81, lon: 720)
Coordinates:
  * time     (time) datetime64[ns] 1991-01-01 ... 1991-12-31T21:00:00
  * lat      (lat) float64 -20.0 -19.5 -19.0 -18.5 -18.0 ... 18.5 19.0 19.5 20.0
  * lon      (lon) float64 -180.0 -179.5 -179.0 -178.5 ... 178.5 179.0 179.5
Data variables:
    Kelv     (time, lat, lon) float64 ...

In [763]:
# #Control which months to track waves. If you wish to track whole year, simply comment out this code.
# def is_amj(month):
#     return  (month >=3)
# claus = claus.sel(time=is_amj(claus['time.month']))

In [780]:
#remove midlatitudes
claus = claus.sel(lat=slice(-20,20))

In [781]:
Kelv = claus.Kelv
lat = claus.lat
lon = claus.lon
time = claus.time

In [783]:
# Centroid technique function. Sourced from AEW tracker by Quinton Lawton with permission. 

def general_centroid(x_in, y_in, PV_in, radius, lati, loni, it, exclude=False):
    '''Similar to that used for calculating PV centers, this computes a centroid point for a given field, using a prescribed
    radius as specified in the function input. Can iterate a given number of times: set in=0 for no iterations.
    
    Uses the more computationally efficient (rad_mask) function to test if a point is within a radial distance or not.'''
    #start= time.time()
    
    # What does our lat/lon refer to?
    guess_lat = y_in[lati]
    guess_lon = x_in[loni]
    old_x = guess_lon
    old_y = guess_lat
    
    #Other important stuff
    res = 1
    #box_num = 6/res; #First number is the degrees you want to cut out from the dataset for analysis
    def get_dist_meters(lon, lat):
        earth_circ = 6371*2*np.pi*1000 #earth's circumference in meters
        lat_met = earth_circ/360 #get the number of meters in a degree latitude (ignoring "bulge")
        lat_dist = np.gradient(lat, axis=0)*lat_met
        lon_dist = np.gradient(lon, axis=1)*np.cos(np.deg2rad(lat))*lat_met
        return lon_dist, lat_dist 
    

    #Commence slicing out the correct data
    lon_new = x_in#[lon_slice]
    lat_new = y_in#[lat_slice]
    PV_slice = PV_in#[lat_slice,lon_slice]
    #Get the distance array for later use    
 
    
    #Make a meshed grid
    LON_X, LAT_Y = np.meshgrid(lon_new, lat_new)
    dx, dy = get_dist_meters(LON_X, LAT_Y)
    
    #Finally, output a 1/0 filter for PV within the given radius
    #pv_filt = inner_filter(lat_new, lon_new, guess_lat, guess_lon, radius)
    pv_filt = rad_mask(lati, loni, dx, dy, radius)

    
    PV_mask = PV_slice.copy()
    lon_mask = LON_X.copy()
    lat_mask = LAT_Y.copy()

    PV_mask[~pv_filt] = np.NaN
    lon_mask[~pv_filt] = np.NaN
    PV_mask[~pv_filt] = np.NaN
    
    #Code original to CCKW tracker. Remove any values of opposite sign than the asssumed peak.
    # When CCKWs are small, a peak of opposite sign may be near and the opposite signs can cancel each other out.
    # So we ignore any values of opposite sign to the center we are currently looking for. 
    
    sign = np.sign(np.nansum(PV_mask))
    initialsign = sign
    if np.nansum(PV_mask) < 50:
        if sign == 1: 
            PV_mask = np.where(PV_mask<0, np.NaN, PV_mask)
        elif sign == -1:
            PV_mask = np.where(PV_mask>0, np.NaN, PV_mask)
    #Calculate the center of mass, finally
    x_cent = np.nansum(lon_mask*PV_mask)/np.nansum(PV_mask)
    y_cent = np.nansum(lat_mask*PV_mask)/np.nansum(PV_mask)
    
    
    ## LOOPING THE ITERATIONS ##
    #Now we will iterate over this it-1 times to converge on the true weighted mass. Each time, we will filter based on 
    #the previous answer's "guess/calculation" of where the PV weighted center is.
    pltcount = 0
    signerror = False
    dist_check = 100
    loop_it = 0
    while dist_check >= 25: #While the new point is at least 50km away from old
        if loop_it >= 10:
            break
        
        old_x = x_cent
        old_y = y_cent
        x_centi, n = find_nearest(lon_new, x_cent)      
        y_centi, n = find_nearest(lat_new, y_cent)  
        #print('loop: '+str(loop_it))
        
        try:

            pv_filt = rad_mask(y_centi, x_centi, dx, dy, radius)

        except:

            print(loop_it,'Exception: Edge of boundary. Skipping to next iteration.')
            
            break
        #Mask out (for all three fields) the filtered PV locations
        PV_mask = PV_slice.copy()
        lon_mask = LON_X.copy()
        lat_mask = LAT_Y.copy()
        PV_mask[~pv_filt] = np.NaN
        lon_mask[~pv_filt] = np.NaN
        PV_mask[~pv_filt] = np.NaN

        x_cent = np.nansum(lon_mask*PV_mask)/np.nansum(PV_mask)
        y_cent = np.nansum(lat_mask*PV_mask)/np.nansum(PV_mask) 
        
        x_centi, n = find_nearest(lon_new, x_cent)
        y_centi, n = find_nearest(lat_new, y_cent)
        signtest = np.sign(PV_mask[y_centi, x_centi])
        if signtest != initialsign:
            signerror = True
            
            break

        dist_check = haversine(x_cent, y_cent, old_x, old_y)
        loop_it = loop_it + 1

    if np.abs(PV_mask[y_centi, x_centi]) <= 1:
        signerror = True
    #Sometimes, if opposite signs are too close together, it can cancel out the maximum. 
    #If this happens, remove all values of the opposite sign and try again. 
    #Don't want to do this every time, as it sometimes puts the center in areas of high gradient
    if signerror: 
        
        pltcount = 0
        dist_check = 100
        loop_it = 0
        
        pv_filt = rad_mask(lati, loni, dx, dy, radius)
        PV_mask = PV_slice.copy()
        lon_mask = LON_X.copy()
        lat_mask = LAT_Y.copy()

        PV_mask[~pv_filt] = np.NaN
        lon_mask[~pv_filt] = np.NaN
        PV_mask[~pv_filt] = np.NaN
        sign = np.sign(np.nansum(PV_mask))
        if sign == 1: 
            PV_mask = np.where(PV_mask<0, np.NaN, PV_mask)
        elif sign == -1:
            PV_mask = np.where(PV_mask>0, np.NaN, PV_mask)
        x_cent = np.nansum(lon_mask*PV_mask)/np.nansum(PV_mask)
        y_cent = np.nansum(lat_mask*PV_mask)/np.nansum(PV_mask)
        
        while dist_check >= 25: #While the new point is at least 50km away from old
            if loop_it >= 10:
                break
            #print(str(loop_it), 'sign error')
            old_x = x_cent
            old_y = y_cent
            x_centi, n = find_nearest(lon_new, x_cent)      
            y_centi, n = find_nearest(lat_new, y_cent)  
            #print('loop: '+str(loop_it))

            try:

                pv_filt = rad_mask(y_centi, x_centi, dx, dy, radius)

            except:

                print(loop_it,'Exception: Edge of boundary. Skipping to next iteration.')
                break
            #Mask out (for all three fields) the filtered PV locations
            PV_mask = PV_slice.copy()
            lon_mask = LON_X.copy()
            lat_mask = LAT_Y.copy()
            PV_mask[~pv_filt] = np.NaN
            lon_mask[~pv_filt] = np.NaN
            PV_mask[~pv_filt] = np.NaN

            sign = np.sign(np.nansum(PV_mask))
            if sign == 1: 
                PV_mask = np.where(PV_mask<0, np.NaN, PV_mask)
            elif sign == -1:
                PV_mask = np.where(PV_mask>0, np.NaN, PV_mask)
            x_cent = np.nansum(lon_mask*PV_mask)/np.nansum(PV_mask)
            y_cent = np.nansum(lat_mask*PV_mask)/np.nansum(PV_mask) 

            x_centi, n = find_nearest(lon_new, x_cent)
            y_centi, n = find_nearest(lat_new, y_cent)
            signtest = np.sign(PV_mask[y_centi, x_centi])
            #print(old_x, old_y, x_cent, y_cent)
            dist_check = haversine(x_cent, y_cent, old_x, old_y)
            loop_it = loop_it + 1
            
    
    #end = time.time()
    #print(str(end-start)+' secs')
    return lon_new, lat_new, pv_filt, PV_mask, x_cent, y_cent

In [784]:
# -----------------------------------------------------
#### MASS CENTROID SETTINGS ####
# -----------------------------------------------------

# Minimum and maximum latitude, over which the Kelvin OLR/Tb is averaged to find the 'first guess.'
# Due to the centroid technique, CCKW centers outside these latitudes can be located, but are less likely.
minlat = -5
maxlat = 10

#Find the latitude halfway between the min and max given above
midlat = (minlat+maxlat)/2
midx = find_nearest(lat,midlat)

# Find the index of the min/max lat
minlati = find_nearest(lat,minlat)[0]
maxlati = find_nearest(lat,maxlat)[0]



# A curvature voticity centroid is run on local maxima before distance testing and either CCKW initation or adding of points.
centroid_rad = 2000 #Radius to weight centroid 

dellat = haversine(0,midx[1],1,midx[1]) #calc distance between longitude at middle latitude
centroid_deg = centroid_rad/dellat      #convert centroid distance to degrees, estimated at the middle latitude

# Distance between current timestep and previous timestep for the tracker to consider both of those points 
# part of the same CCKW. 
samewave_deg = 5 #degrees

#time under which wave is removed, i.e. wave must exist for this length of time to be considered a Kelvin Wave
tooshort = 3 #days

#number of observations per day in the dataset
obsperday = 8

# Calculate horizontal resolution
deg0arg = np.where(lon.values==0)[0][0]
deg1arg = np.where(lon.values==1)[0][0]
obsperlon = deg1arg-deg0arg


In [785]:
#Set minimum threshold on Hovmoller to be considered a 'peak'
#CCKW will not be tracked if it does not meet this threshold.

std = 4.82
halfstd = std/2

std = halfstd #K

In [786]:
#Select how far apart Hovmoller peaks must be to be considered seperate waves.
halfwavelength = 1400 #km
perdegree = haversine(0,midx[1],1,midx[1])
halfwavelengthlon = halfwavelength/perdegree
dis = halfwavelengthlon * obsperlon #number of obs to the left or right to check for peaks
#for more than one peak less than this number of obs apart, code will only choose the higher peak

In [787]:
timesince = np.datetime64('1900-01-01T00:00:00')  #time in hours since 1900-01-01

#Create arrays that will contain the final waves
#First dimension is the wave index (nth tracked wave)
#Second dimension is the time index (nth timestep)
#If wave does not exist at that timestep, it will be filled with NaNs
#This is so the arrays are of a uniform size, as we do not know how long each wave will be
wavelat = np.full((1,len(time)),np.nan)
wavelon = np.full((1,len(time)),np.nan)
wavevalue = np.full((1,len(time)),np.nan)
wavedate = np.full((1,len(time)),np.nan)
newwave = np.full((1,len(time)),np.nan)
#newwave is an empty array that we will concatenate onto the end of each wave array when a new wave is tracked
#This is because we do not know how many waves we will track from the beginning

# Set run counter to zero
count =0


timelen, = time.shape
#Run through every timestep
for t in range(0,timelen):
    #print(time[t])

    #print(str(t)+' out of '+str(len(time)))
    #FIRST -- TAKE A SLICE OUT OF DATA FOR THE CURRENT TIMESTEP
    kelv_data = Kelv[t, :, :]
    #Create a 'Hovmoller' at a single timestep. Take the mean between the min and max lat.
    hov = kelv_data.sel(lat=slice(minlat,maxlat)).mean(dim='lat')
    #Find the peaks (both positive and negative)
    hov_extrema = np.concatenate((scipy.signal.find_peaks(hov, height=std, distance=dis)[0],scipy.signal.find_peaks(-hov, height=std, distance=dis)[0]))
    hov_extremavalues = hov[hov_extrema]
    lentotal = len(hov_extrema)
    
    #Create DataFrame with the location and value of each peak, then sort by longitude
    pd_extrema = pd.DataFrame({'index':lon[hov_extrema],'lon':lon[hov_extrema],'values':hov_extremavalues})
    pd_extrema = pd_extrema.set_index('index')
    pd_extrema = pd_extrema.sort_values(by='lon')
    testlats = []
    testlons = [] 
    
    #--- TEST FOR TWO IMPORTANT EXCEPTIONS -----------------------
    # -- FIRST ITERATION or
    # -- NO INITIATED WAVES EXIST
    # -- For these, we only care about trying to designate "new" waves
    if (t == 0) | (count==0): #This only runs if an array has not already been created
        #for loop over each extrema in this timestep
        for i in range(0,pd_extrema.shape[0]):
            
            

            #Last wave will always be empty. We will delete this later
            wavelat = np.concatenate((wavelat,newwave), axis=0)
            wavelon = np.concatenate((wavelon,newwave), axis=0)
            wavevalue = np.concatenate((wavevalue,newwave), axis=0)
            wavedate = np.concatenate((wavedate,newwave), axis=0)
          
            #loni is the longitude integer of this particular extrema. This will be our first guess of the CCKW center
            pdloc = pd_extrema.iloc[i]
            loni = pdloc.loc['lon']
            
            latshape, = lat.shape
            lonshape, = lon.shape
            #handle extrema near data boundaries by extending past 0 and 360 degrees. 
            extend = np.ceil(180*obsperlon)
            extend = int(extend)
            kelv_data_ex = np.zeros((latshape, lonshape+(extend*2)))
            lon_ex = np.zeros((lonshape+(extend*2)))
            kelv_data_ex[:,0:extend] = kelv_data[:,-extend:]
            kelv_data_ex[:,-extend:] = kelv_data[:,0:extend]
            kelv_data_ex[:,extend:(-extend)] = kelv_data[:,:]
            lon_ex[-extend:] = lon[0:extend] + 360
            lon_ex[extend:(-extend)] = lon[:]
            lon_ex[0:extend] = lon[-extend:] - 360
            loni = find_nearest(lon_ex,loni)[0]
            
            #Make a latitude first guess by averaging filtered Tb over several longitudes and finding the maximum
            lonmean = np.mean(kelv_data_ex[minlati:maxlati,(loni-8):(loni+8)], axis=1)
            latmeanmax = np.max(lonmean)
            latmeanmaxi = find_nearest(lat.values,latmeanmax)[0]
            #Run the centroid technique using our previous first guess
            centtest = general_centroid(lon_ex, lat.values, kelv_data_ex, centroid_rad, latmeanmaxi, loni, 0, exclude=True)
            #if centroid longitude results are outside of 0-359, convert.
            if centtest[-2] < 0: 
                lontemp = 360 + centtest[-2]
                lonw = find_nearest(lon, lontemp)
            elif centtest[-2] > 360:
                lontemp = centtest[-2]-360
                lonw = find_nearest(lon, lontemp)
            else:
                lonw = find_nearest(lon,centtest[-2])
            latw = find_nearest(lat,centtest[-1])
            wavelat[i,t] = latw[1]
            if lonw[1] >= 360:
                lontemp = lonw[1]-360
                lonw = find_nearest(lon, lontemp)
            elif lonw[1] < 0:
                lontemp = lonw[1]+360
                lonw = find_nearest(lon, lontemp)
            elif lonw[1] == 360.:
                lonw = [0,0.]
            else: 
                lontemp = lonw[1]
                lonw = find_nearest(lon, lontemp)
                
            #Since this is the first timestep...start a wave.
            wavelon[i,t] = lonw[1]
            wavevalue[i,t] = kelv_data.values[latw[0],lonw[0]]
            wavedate[i,t] = np.timedelta64(time[t].values-timesince, 'h')/np.timedelta64(1,'h')

            count = count+1
    else:
        #for loop over each extrema in this timestep
        for i in range(0,pd_extrema.shape[0]):
            
            #loni is the longitude integer of this particular extrema. This will be our first guess of the CCKW center
            pdloc = pd_extrema.iloc[i]
            loni = pdloc.loc['lon']
            
            
            latshape, = lat.shape
            lonshape, = lon.shape
            #handle extrema near data boundaries by extending past 0 and 360 degrees. 
            extend = np.ceil(180*obsperlon)
            extend = int(extend)
            kelv_data_ex = np.zeros((latshape, lonshape+(extend*2)))
            lon_ex = np.zeros((lonshape+(extend*2)))
            kelv_data_ex[:,0:extend] = kelv_data[:,-extend:]
            kelv_data_ex[:,-extend:] = kelv_data[:,0:extend]
            kelv_data_ex[:,extend:(-extend)] = kelv_data[:,:]

            lon_ex[-extend:] = lon[0:extend] + 360
            lon_ex[extend:(-extend)] = lon[:]
            lon_ex[0:extend] = lon[-extend:] - 360
            loni = find_nearest(lon_ex,loni)[0]
            #Make a latitude first guess by averaging filtered Tb over several longitudes and finding the maximum
            lonmean = np.mean(kelv_data_ex[minlati:maxlati,(loni-8):(loni+8)], axis=1)
            latmeanmax = np.max(lonmean)
            
            latmeanmaxi = find_nearest(lat.values,latmeanmax)[0]
            #Run the centroid technique using our previous first guess
            centtest = general_centroid(lon_ex, lat.values, kelv_data_ex, centroid_rad, latmeanmaxi, loni, 0, exclude=True)
            
            latw = find_nearest(lat,centtest[-1])
            
            #if centroid longitude results are outside of 0-359, convert.
            if centtest[-2] < 0: 
                lontemp = 360 + centtest[-2]
                lonw = find_nearest(lon, lontemp)
            elif centtest[-2] > 360:
                lontemp = centtest[-2]-360
                lonw = find_nearest(lon, lontemp)
            else:
                lonw = find_nearest(lon,centtest[-2])
            
            
            #Test if wave is within certain distance of a wave from the previous timestep and has the same sign
            
            #create boolean to test if previous timestep has the same sign
            try:
                testloc1 = np.nanargmin(np.sqrt((lonw[1]-wavelon[:, t-1])**2+(latw[1]-wavelat[:,t-1])**2))
                testsign1 = bool(np.sign(kelv_data.values[latw[0],lonw[0]]) == np.sign(wavevalue[testloc1,t-1]))
            except:
                testsign1 = False
                
            try:
                testloc2 = np.nanargmin((lonw[1]+360)-wavelon[:,t-1])
                testsign2 = bool(np.sign(kelv_data.values[latw[0],lonw[0]]) == np.sign(wavevalue[testloc2,t-1]) and lonw[1]<samewave_deg)
            except:
                testsign2 = False
                
                
            #If wave is within a certain distance of an extrema in the previous timestep and has the same sign, 
            #consider it the same wave
            if np.nanmin(np.sqrt((lonw[1]-wavelon[:, t-1])**2+(latw[1]-wavelat[:,t-1])**2)) < samewave_deg and testsign1:
                
                iloc = np.nanargmin(np.sqrt((lonw[1]-wavelon[:, t-1])**2+(latw[1]-wavelat[:,t-1])**2))
                wavelat[iloc,t] = latw[1]
                #convert back to 0-359 if necessary
                if lonw[1] >= 360:
                    lontemp = lonw[1]-360
                    lonw = find_nearest(lon, lontemp)
                elif lonw[1] < 0:
                    lontemp = lonw[1]+360
                    lonw = find_nearest(lon, lontemp)
                elif lonw[1] == 360.:
                    lonw = [0,0.]
                else: 
                    lontemp = lonw[1]
                    lonw = find_nearest(lon, lontemp)
                #add data to wave
                wavelon[iloc,t] = lonw[1]
                wavevalue[iloc,t] = kelv_data.values[latw[0],lonw[0]]
                wavedate[iloc,t] = np.timedelta64(time[t].values-timesince, 'h')/np.timedelta64(1,'h')
            
            #if data is near the boundary and is within a certain distance of an extrema in the previous
            #timestep and has same sign, consider them the same wave
            elif np.nanmin((lonw[1]+360)-wavelon[:,t-1]) < samewave_deg and lonw[1] < samewave_deg and testsign2:
                
                iloc = np.nanargmin((lonw[1]+360)-wavelon[:,t-1])
                
                wavelat[iloc,t] = latw[1]
                wavevalue[iloc,t] = kelv_data.values[latw[0],lonw[0]]
                wavedate[iloc,t] = np.timedelta64(time[t].values-timesince, 'h')/np.timedelta64(1,'h')
                #convert back to 0-359 if necessary
                if lonw[1] >= 360:
                    lontemp = lonw[1]-360
                    lonw = find_nearest(lon, lontemp)
                elif lonw[1] < 0:
                    lontemp = lonw[1]+360
                    lonw = find_nearest(lon, lontemp)
                elif lonw[1] == 360.:
                    lonw = [0,0.]
                else: 
                    lontemp = lonw[1]
                    lonw = find_nearest(lon, lontemp)
                
                wavelon[iloc,t] = lonw[1]
            else:
                #make new wave
                #remember, last wave always empty, so new wave will be the second to last
                wavelat = np.concatenate((wavelat,newwave), axis=0)
                wavelon = np.concatenate((wavelon,newwave), axis=0)
                wavevalue = np.concatenate((wavevalue,newwave), axis=0)
                wavedate = np.concatenate((wavedate,newwave), axis=0)
                
                if lonw[1] >= 360:
                    lontemp = lonw[1]-360
                    lonw = find_nearest(lon, lontemp)
                elif lonw[1] < 0:
                    lontemp = lonw[1]+360
                    lonw = find_nearest(lon, lontemp)
                elif lonw[1] == 360.:
                    lonw = [0,0.]
                else: 
                    lontemp = lonw[1]
                    lonw = find_nearest(lon, lontemp)
                
                wavelon[-2,t] = lonw[1]
                wavelat[-2,t] = latw[1]
                wavevalue[-2,t] = kelv_data.values[latw[0],lonw[0]]
                wavedate[-2,t] = np.timedelta64(time[t].values-timesince, 'h')/np.timedelta64(1,'h')

            count = count+1
            


0 Exception: Edge of boundary. Skipping to next iteration.
0 Exception: Edge of boundary. Skipping to next iteration.


In [788]:
# Wave Cleanup

wavenumber,tlen = wavelon.shape

#remove empty waves
for i in range(0,wavenumber):
    if np.isnan(wavelon[i,:].all()):
        wavelon = np.delete(wavelon, i, axis=0)
        wavelat = np.delete(wavelat, i, axis=0)
        wavevalue = np.delete(wavevalue, i, axis=0)
        wavedate = np.delete(wavedate,i,axis=0)
        
wavenumber,tlen = wavelon.shape

#remove duplicates         
for i in range(0,wavenumber):
    if np.all(wavelon[i,:] == wavelon[i-1,:]):
        wavelon = np.delete(wavelon, i, axis=0)
        wavelat = np.delete(wavelat, i, axis=0)
        wavevalue = np.delete(wavevalue, i, axis=0)
        wavedate = np.delete(wavedate,i,axis=0)  
           

wavenumber,tlen = wavelat.shape

countdif = 1
#remove waves that exist for less than a certain length of time
while countdif > 0:
    wavenumber,tlen = wavelat.shape
    tempnum = wavenumber
    for i in np.arange(0,wavenumber):
        try: 
            templon = wavelon[i,:][~np.isnan(wavelon[i,:])]
            #print(templon)
            if templon.size < (tooshort*obsperday):
                wavelon = np.delete(wavelon, i, axis=0)
                wavelat = np.delete(wavelat, i, axis=0)
                wavevalue = np.delete(wavevalue, i, axis=0)
                wavedate = np.delete(wavedate,i,axis=0) 
        except:
            continue
    wavenumber,tlen = wavelat.shape
    countdif = tempnum - wavenumber
    print(tempnum,wavenumber,countdif)
wavenumber,tlen = wavelat.shape    
#if first value and last value have different sign, remove
#prevents center migration to opposite phase
countdif = 1

while countdif > 0:
    wavenumber,tlen = wavelat.shape
    tempnum = wavenumber
    for i in np.arange(0,wavenumber):
        try: 
            templon = wavelon[i,:][~np.isnan(wavelon[i,:])]
            #print(templon)
            if np.sign(templon[i,0]) != np.sign(templon[i,-1]):
                wavelon = np.delete(wavelon, i, axis=0)
                wavelat = np.delete(wavelat, i, axis=0)
                wavevalue = np.delete(wavevalue, i, axis=0)
                wavedate = np.delete(wavedate,i,axis=0) 
        except:
            continue
    wavenumber,tlen = wavelat.shape
    countdif = tempnum - wavenumber
    print(tempnum,wavenumber,countdif)
wavenumber,tlen = wavelat.shape    

2162 1175 987
1175 683 492
683 449 234
449 361 88
361 343 18
343 343 0
343 343 0


In [773]:
# Create NETCDF4 dataset
ncout = n.Dataset('/thorncroftlab_rit/kd829281/Data/KelvinTracksTest/KelvinTracksNewLats'+year+'.nc', 'w', format='NETCDF4')

maxN, timelen = np.shape(wavelon)
Nrange = np.arange(1,maxN+1)
timeInt = (time.values-timesince)/np.timedelta64(1,'h')

ncout.createDimension('waveN', maxN)  
ncout.createDimension('time', timelen)

timec = ncout.createVariable('time', np.float64, ('time',))
timec.units = 'hours since 1900-01-01 00:00:00'
timec.calendar = 'standard'

wavec = ncout.createVariable('waveN', np.float64, ('waveN',))
wavec.long_name = 'nth tracked wave'

# create time axis
wavetimec = ncout.createVariable('wavetime', np.float64, ('waveN', 'time'))
wavetimec.long_name = 'time associated with wave location'
wavetimec.units = 'hours since 1900-01-01 00:00:00'
wavetimec.calendar = 'standard'


# create latitude axis
latc = ncout.createVariable('wavelat', np.float64, ('waveN', 'time'))
latc.standard_name = 'lat'
latc.long_name = 'latitude'
latc.units = 'degrees_north'


# create longitude axis
lonc = ncout.createVariable('wavelon', np.float64, ('waveN', 'time'))
lonc.standard_name = 'longitude'
lonc.long_name = 'longitude'
lonc.units = 'degrees_east'


# create variable array
vout = ncout.createVariable('wavevalue', np.float64, ('waveN', 'time'))
vout.long_name = 'Kelvin-filtered b  rightness temperature (at wave center)'
vout.units = 'K'

# copy axis from original dataset
wavec[:] = Nrange[:] # broken
timec[:] = timeInt[:]
lonc[:] = wavelon[:]
latc[:] = wavelat[:]
vout[:] = wavevalue[:]
wavetimec[:] = wavedate[:]

# close files
ncout.close()